<a href="https://colab.research.google.com/github/meenakshi3151/CSN14403/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Implement a Python program to solve the 8-Puzzle problem using Depth-First Search. Include options to choose different initial and goal states. Output the sequence of moves to reach the goal state.

In [ ]:
import sys
import numpy as np


class Node:
	def __init__(self, state, parent, action):
		self.state = state
		self.parent = parent
		self.action = action


class StackFrontier:
	def __init__(self):
		self.frontier = []

	def add(self, node):
		self.frontier.append(node)

	def contains_state(self, state):
		return any((node.state[0] == state[0]).all() for node in self.frontier)

	def empty(self):
		return len(self.frontier) == 0

	def remove(self):
		if self.empty():
			raise Exception("Empty Frontier")
		else:
			node = self.frontier[-1]
			self.frontier = self.frontier[:-1]
			return node


class QueueFrontier(StackFrontier):
	def remove(self):
		if self.empty():
			raise Exception("Empty Frontier")
		else:
			node = self.frontier[0]
			self.frontier = self.frontier[1:]
			return node


class Puzzle:
	def __init__(self, start, startIndex, goal, goalIndex):
		self.start = [start, startIndex]
		self.goal = [goal, goalIndex]
		self.solution = None

	def neighbors(self, state):
		mat, (row, col) = state
		results = []

		if row > 0:
			mat1 = np.copy(mat)
			mat1[row][col] = mat1[row - 1][col]
			mat1[row - 1][col] = 0
			results.append(('up', [mat1, (row - 1, col)]))
		if col > 0:
			mat1 = np.copy(mat)
			mat1[row][col] = mat1[row][col - 1]
			mat1[row][col - 1] = 0
			results.append(('left', [mat1, (row, col - 1)]))
		if row < 2:
			mat1 = np.copy(mat)
			mat1[row][col] = mat1[row + 1][col]
			mat1[row + 1][col] = 0
			results.append(('down', [mat1, (row + 1, col)]))
		if col < 2:
			mat1 = np.copy(mat)
			mat1[row][col] = mat1[row][col + 1]
			mat1[row][col + 1] = 0
			results.append(('right', [mat1, (row, col + 1)]))

		return results

	def print(self):
		solution = self.solution if self.solution is not None else None
		print("Start State:\n", self.start[0], "\n")
		print("Goal State:\n",  self.goal[0], "\n")
		print("\nStates Explored: ", self.num_explored, "\n")
		print("Solution:\n ")
		for action, cell in zip(solution[0], solution[1]):
			print("action: ", action, "\n", cell[0], "\n")
		print("Goal Reached!!")

	def does_not_contain_state(self, state):
		for st in self.explored:
			if (st[0] == state[0]).all():
				return False
		return True

	def solve(self):
		self.num_explored = 0

		start = Node(state=self.start, parent=None, action=None)
		frontier = QueueFrontier()
		frontier.add(start)

		self.explored = []

		while True:
			if frontier.empty():
				raise Exception("No solution")

			node = frontier.remove()
			self.num_explored += 1

			if (node.state[0] == self.goal[0]).all():
				actions = []
				cells = []
				while node.parent is not None:
					actions.append(node.action)
					cells.append(node.state)
					node = node.parent
				actions.reverse()
				cells.reverse()
				self.solution = (actions,  cells)
				return

			self.explored.append(node.state)

			for action, state in self.neighbors(node.state):
				if not frontier.contains_state(state) and self.does_not_contain_state(state):
					child = Node(state=state, parent=node, action=action)
					frontier.add(child)


start = np.array([[1, 2, 3], [8, 0, 4], [7, 6, 5]])
goal = np.array([[2, 8, 1], [0, 4, 3], [7, 6, 5]])


startIndex = (1, 1)
goalIndex = (1, 0)


p = Puzzle(start, startIndex, goal, goalIndex)
p.solve()
p.print()

Start State:
 [[1 2 3]
 [8 0 4]
 [7 6 5]] 

Goal State:
 [[2 8 1]
 [0 4 3]
 [7 6 5]] 


States Explored:  358 

Solution:
 
action:  up 
 [[1 0 3]
 [8 2 4]
 [7 6 5]] 

action:  left 
 [[0 1 3]
 [8 2 4]
 [7 6 5]] 

action:  down 
 [[8 1 3]
 [0 2 4]
 [7 6 5]] 

action:  right 
 [[8 1 3]
 [2 0 4]
 [7 6 5]] 

action:  right 
 [[8 1 3]
 [2 4 0]
 [7 6 5]] 

action:  up 
 [[8 1 0]
 [2 4 3]
 [7 6 5]] 

action:  left 
 [[8 0 1]
 [2 4 3]
 [7 6 5]] 

action:  left 
 [[0 8 1]
 [2 4 3]
 [7 6 5]] 

action:  down 
 [[2 8 1]
 [0 4 3]
 [7 6 5]] 

Goal Reached!!


2. Write a Python program that solves the 8-Puzzle using Breadth-First Search. Allow the user to input different initial and goal states. Output the shortest sequence of moves to reach the goal.

In [ ]:
from collections import deque

def print_board(board):
    for row in board:
        print(" ".join(str(cell) if cell != 0 else '*' for cell in row))

def find_blank(board):
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:
                return i, j

def move_up(board):
    i, j = find_blank(board)
    if i > 0:
        board[i][j], board[i - 1][j] = board[i - 1][j], board[i][j]
        return True
    return False

def move_down(board):
    i, j = find_blank(board)
    if i < 2:
        board[i][j], board[i + 1][j] = board[i + 1][j], board[i][j]
        return True
    return False

def move_left(board):
    i, j = find_blank(board)
    if j > 0:
        board[i][j], board[i][j - 1] = board[i][j - 1], board[i][j]
        return True
    return False

def move_right(board):
    i, j = find_blank(board)
    if j < 2:
        board[i][j], board[i][j + 1] = board[i][j + 1], board[i][j]
        return True
    return False

def apply_move(board, move):
    if move == 'U':
        return move_up(board)
    elif move == 'D':
        return move_down(board)
    elif move == 'L':
        return move_left(board)
    elif move == 'R':
        return move_right(board)

def bfs(initial, goal):
    visited = set()
    queue = deque([(initial, [])])

    while queue:
        current_state, path = queue.popleft()
        if current_state == goal:
            return path

        if current_state not in visited:
            visited.add(current_state)
            for move in ['U', 'D', 'L', 'R']:
                new_state = [row.copy() for row in current_state]
                if apply_move(new_state, move):
                    queue.append((new_state, path + [move]))

    return None

if __name__ == "__main__":
    print("Enter the initial state (each row as space-separated values):")
    initial_state = [list(map(int, input().split())) for _ in range(3)]

    print("\nEnter the goal state (each row as space-separated values):")
    goal_state = [list(map(int, input().split())) for _ in range(3)]

    print("\nInitial State:")
    print_board(initial_state)
    print("\nGoal State:")
    print_board(goal_state)

    result = bfs(initial_state, goal_state)

    if result is not None:
        print(f"\nShortest sequence of moves to reach the goal: {' '.join(result)}")
    else:
        print("\nGoal not reachable.")


3. Implement a Python program to solve the 8-Puzzle using Uniform Cost Search. Allow users to input different initial and goal states, along with the cost of each move. Output the optimal sequence of moves and the cost of reaching the goal.

In [4]:
# goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
goal_state = [2, 5, 1, 3, 6, 0, 4, 8, 7]
# goal_state = [3, 2, 1, 4, 5, 6, 8, 0, 7]

counter = 0


def display_board(state):
    print("-------------")
    print("| %i | %i | %i |" % (state[0], state[3], state[6]))
    print("-------------")
    print("| %i | %i | %i |" % (state[1], state[4], state[7]))
    print("-------------")
    print("| %i | %i | %i |" % (state[2], state[5], state[8]))
    print("-------------")


def move_up(state):
    """Moves the blank tile up on the board. Returns a new state as a list."""

    new_state = state[:]
    index = new_state.index(0)

    if index not in [0, 3, 6]:

        temp = new_state[index - 1]
        new_state[index - 1] = new_state[index]
        new_state[index] = temp
        return new_state
    else:

        return None


def move_down(state):
    """Moves the blank tile down on the board. Returns a new state as a list."""
    # Perform object copy
    new_state = state[:]
    index = new_state.index(0)
    # Sanity check
    if index not in [2, 5, 8]:
        # Swap the values.
        temp = new_state[index + 1]
        new_state[index + 1] = new_state[index]
        new_state[index] = temp
        return new_state
    else:
        # Can't move, return None.
        return None


def move_left(state):
    """Moves the blank tile left on the board. Returns a new state as a list."""
    new_state = state[:]
    index = new_state.index(0)
    # Sanity check
    if index not in [0, 1, 2]:
        # Swap the values.
        temp = new_state[index - 3]
        new_state[index - 3] = new_state[index]
        new_state[index] = temp
        return new_state
    else:
        # Can't move it, return None
        return None


def move_right(state):
    """Moves the blank tile right on the board. Returns a new state as a list."""
    # Performs an object copy. Python passes by reference.
    new_state = state[:]
    index = new_state.index(0)
    # Sanity check
    if index not in [6, 7, 8]:
        # Swap the values.
        temp = new_state[index + 3]
        new_state[index + 3] = new_state[index]
        new_state[index] = temp
        return new_state
    else:
        # Can't move, return None
        return None


def create_node(state, parent, operator, depth, cost):
    return Node(state, parent, operator, depth, cost)


def expand_node(node):
    """Returns a list of expanded nodes"""
    expanded_nodes = [create_node(move_up(node.state), node, "u", node.depth + 1, 0),
                      create_node(move_down(node.state), node, "d", node.depth + 1, 0),
                      create_node(move_left(node.state), node, "l", node.depth + 1, 0),
                      create_node(move_right(node.state), node, "r", node.depth + 1, 0)]

    expanded_nodes = [node for node in expanded_nodes if node.state is not None]
    return expanded_nodes


def uniform_cost(start, goal):
    start_node = create_node(start, None, None, 0, 0)
    fringe = []
    path = []
    fringe.append(start_node)
    current = fringe.pop(0)
    while current.state != goal:
        temp = expand_node(current)
        for item in temp:
            item.depth += current.depth
            fringe.append(item)
        fringe.sort(key=lambda x: x.depth)
        current = fringe.pop(0)
    while current.parent is not None:
        path.insert(0, current.operator)
        current = current.parent
    return path


def h(state, goal):
    dmatch = 0
    for i in range(0, 9):
        if state.state[i] != goal[i]:
            dmatch += 1
    state.heuristic = dmatch



class Node:
    def __init__(self, state, parent, operator, depth, cost):

        self.state = state

        self.parent = parent

        self.operator = operator

        self.depth = depth

        self.cost = cost

        self.heuristic = None


def readfile(filename):
    f = open(filename)
    data = f.read()

    data = data.strip("\n")

    data = data.split(" ")
    state = []
    for element in data:
        state.append(int(element))
    print('state: ', state)
    return state


# Main method
def main():
    starting_state = [3, 2, 1, 4, 5, 6, 8, 7, 0]

    result = uniform_cost(starting_state, goal_state)

    if result is None:
        print("No solution found")
    elif result == [None]:
        print("Start node was the goal!")
    else:
        print(result)
        print(len(result), " moves")


if __name__ == "__main__":
    main()

['u', 'u', 'l', 'l', 'd', 'r', 'd']
7  moves


4. Write a Python program to solve the 8-Puzzle problem using Bidirectional Search. Allow users to specify different initial and goal states. Output the shortest sequence of moves and the meeting point of the two searches.

In [ ]:
from collections import deque
import heapq
import numpy as np
import random
import sys

N = 3

def Manhattan(s1, s2):
  dist = 0
  for i1, d in enumerate(s1):
    i2 = s2.index(d)
    x1 = i1 % N
    y1 = i1 // N
    x2 = i2 % N
    y2 = i2 // N
    dist += abs(x1 - x2) + abs(y1 - y2)
  return dist

def Hamming(s1, s2):
  return sum(x != y for x, y in zip(s1, s2))

class Node:
  dirs = [0, -1, 0, 1, 0]
  def __init__(self, state, parent = None, h = 0):
    self.state = state
    self.parent = parent
    self.g = parent.g + 1 if parent else 0
    self.h = h
    self.f = self.g + self.h

  def getMoves(self):
    moves = []
    index = self.state.index(0)
    x = index % N
    y = index // N
    for i in range(4):
      tx = x + Node.dirs[i]
      ty = y + Node.dirs[i + 1]
      if tx < 0 or ty < 0 or tx == N or ty == N:
        continue
      i = ty * N + tx
      move = list(self.state)
      move[index] = move[i]
      move[i] = 0
      moves.append(tuple(move))
    return moves

  def print(self):
    print(np.reshape(self.state, (N, N)))

  def __lt__(self, other):
    return self.f < other.f

def AStarSearch(start_state, end_state, heuristic):
  def h(s):
    return heuristic(s, end_state)
  q = []
  s = Node(start_state, h=h(start_state))
  heapq.heappush(q, s)
  opened = {s.state : s.f}
  closed = dict()
  while q:
    n = heapq.heappop(q)
    if n.state == end_state:
      return n, len(opened), len(closed)
    if n.state in closed: continue
    closed[n.state] = n.f
    for move in n.getMoves():
      node = Node(move, n, h(move))
      if move in opened and opened[move] <= node.f: continue
      opened[node.state] = node.f
      heapq.heappush(q, node)
  return None, len(opened), len(closed)

def BFS(start_state, end_state):
  q = deque()
  q.append(Node(start_state))
  opened = set(start_state)
  closed = 0
  while q:
    n = q.popleft()
    if n.state == end_state:
      return n, len(opened), closed
    closed += 1
    for move in n.getMoves():
      if move in opened: continue
      opened.add(move)
      q.append(Node(move, n))
  return None, len(opened), closed

def getRootNode(n):
  return getRootNode(n.parent) if n.parent else n

def BidirectionalBFS(start_state, end_state):
  def constructPath(p, o):
    while o:
      t = o.parent
      o.parent = p
      p, o = o, t
    return p
  ns = Node(start_state)
  ne = Node(end_state)
  q = [deque([ns]), deque([ne])]
  opened = [{start_state : ns}, {end_state: ne}]
  closed = [0, 0]
  while q[0]:
    l = len(q[0])
    while l > 0:
      p = q[0].popleft()
      closed[0] += 1
      for move in p.getMoves():
        n = Node(move, p)
        if move in opened[1]:
          o = opened[1][move]
          if getRootNode(n).state == end_state:
            o, n = n, o
          n = constructPath(n, o.parent)
          return n, len(opened[0]) + len(opened[1]), closed[0] + closed[1]
        if move in opened[0]: continue
        opened[0][move] = n
        q[0].append(n)
      l -= 1
    q.reverse()
    opened.reverse()
    closed.reverse()
  return None, len(opened[0]) + len(opened[1]), closed[0] + closed[1]


def print_path(n):
  if not n: return
  print_path(n.parent)
  n.print()

def solvable(state):
  inv = 0
  for i in range(N*N):
    for j in range(i + 1, N*N):
      if all((state[i] > 0, state[j] > 0, state[i] > state[j])):
        inv += 1
  return inv % 2 == 0

if __name__ == '__main__':
  s = [1, 2, 3, 4, 5, 6, 7, 8, 0]
  e = [1, 2, 3, 4, 5, 6, 7, 0, 8]
  i = 0
  while True:
    random.shuffle(s)
    if not solvable(s): continue
    n1, opened1, closed1 = AStarSearch(tuple(s), tuple(e), heuristic=Manhattan)
    n2, opened2, closed2 = AStarSearch(tuple(s), tuple(e), heuristic=Hamming)
    n3, opened3, closed3 = BFS(tuple(s), tuple(e))
    n4, opened4, closed4 = BidirectionalBFS(tuple(s), tuple(e))
    print("%d\t%d\t%d\t%d" % (closed1, closed2, closed3, closed4))
    sys.stdout.flush()

    i += 1
    if i == 1000: break

5. Implement a Python program for the 8-Puzzle problem using Depth Limited Search (DLS). The program should take an initial and goal state as input, allow the user to set a depth limit, and output the sequence of moves to reach the goal within the specified depth. If the goal is not reachable within the limit, indicate the unsuccessful search.

In [9]:
from collections import deque

def print_board(board):
    for row in board:
        print(" ".join(str(cell) if cell != 0 else '*' for cell in row))

def find_blank(board):
    for i in range(3):
        for j in range(3):
            if board[i][j] == 0:
                return i, j

def move_up(board):
    i, j = find_blank(board)
    if i > 0:
        board[i][j], board[i - 1][j] = board[i - 1][j], board[i][j]
        return True
    return False

def move_down(board):
    i, j = find_blank(board)
    if i < 2:
        board[i][j], board[i + 1][j] = board[i + 1][j], board[i][j]
        return True
    return False

def move_left(board):
    i, j = find_blank(board)
    if j > 0:
        board[i][j], board[i][j - 1] = board[i][j - 1], board[i][j]
        return True
    return False

def move_right(board):
    i, j = find_blank(board)
    if j < 2:
        board[i][j], board[i][j + 1] = board[i][j + 1], board[i][j]
        return True
    return False

def apply_move(board, move):
    if move == 'U':
        return move_up(board)
    elif move == 'D':
        return move_down(board)
    elif move == 'L':
        return move_left(board)
    elif move == 'R':
        return move_right(board)

def dls(initial, goal, depth_limit):
    if initial == goal:
        return []

    visited = set()
    stack = deque([(initial, [])])

    while stack:
        current_state, path = stack.pop()
        if current_state == goal:
            return path

        if len(path) < depth_limit:
            for move in ['U', 'D', 'L', 'R']:
                new_state = [row.copy() for row in current_state]
                if apply_move(new_state, move):
                    new_state_tuple = tuple(map(tuple, new_state))
                    if new_state_tuple not in visited:
                        stack.append((new_state, path + [move]))
                        visited.add(new_state_tuple)

    return None

if __name__ == "__main__":
    initial_state = [[1, 2, 3], [4, 0, 5], [6, 7, 8]]
    goal_state = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]

    print("Initial State:")
    print_board(initial_state)
    print("\nGoal State:")
    print_board(goal_state)

    depth_limit = int(input("Enter the depth limit: "))
    result = dls(initial_state, goal_state, depth_limit)

    if result is not None:
        print(f"\nMoves to reach the goal within depth limit ({depth_limit}): {' '.join(result)}")
    else:
        print(f"\nGoal not reachable within depth limit ({depth_limit}).")


Initial State:
1 2 3
4 * 5
6 7 8

Goal State:
* 1 2
3 4 5
6 7 8
Enter the depth limit: 100

Moves to reach the goal within depth limit (100): R D L L U R R D L L U R R D L L U R R D L L U R R D L U R D L L U R R D L L U R R U L L D R D R U U L L D R D R U L L U R D R D L L U U R R D D L L U R D L U R R U L D L U R R D D L L U U
